In [ ]:
!pip3 install tweepy datetime

In [ ]:
!python3 get_tweets.py

In [ ]:
import json

In [ ]:
# Abrir arquivo e ler linhas
with open('./collected_tweets_2021-03-09-22-16-43.txt', 'r') as read_file:
    tweets = read_file.readlines()

In [ ]:
print(len(tweets))
print(type(tweets))

In [ ]:
# Parseia o json de tweet e exporta em arquiv .json
with open('tweet.json', 'w') as json_file:
    json.dump(
        json.loads(json.loads(tweets[0])),
        json_file
    )

In [ ]:
type(json.loads(tweets[0]))

In [ ]:
# Parseia o json 2x. Transformando a string json para um dicionario
json.loads(
    json.loads(tweets[0])
)

In [ ]:
type(
    json.loads(
    json.loads(tweets[0])
    )
)

In [ ]:
# Utiliza uma lista compreensiva para pegar todos os tweets já parseados
# se lê retorne um tweet (um json) parseado para cada posição em tweets (uma lista)
parsed_tweets = [json.loads(json.loads(i)) for i in tweets]

In [ ]:
parsed_tweets[10]

In [ ]:
len(parsed_tweets)

# Acessando as chaves do JSON (DICIONARIO)

In [ ]:
primeiro_tweet = parsed_tweets[0]
primeiro_tweet

In [ ]:
primeiro_tweet["created_at"]

In [ ]:
type(primeiro_tweet["truncated"])

In [ ]:
list(primeiro_tweet.items())[1]

In [ ]:
# Acessa uma chave do json de primeiro nivel (root)
primeiro_tweet["text"]

In [ ]:
# Acessa uma chave user (um novo dicionario)
primeiro_tweet['user']['id']

In [ ]:
# Posso acessar assim muitos aninhamentos de jsons
primeiro_tweet['extended_tweet']['entities']['user_mentions']

In [ ]:
# Filtra apenas os tweets com retweet
# Retorne um tweet para cada tweet em parsed_tweets (array de tweets) se retweeted_status estiver no tweet
#Se ler do fim ao começo da expressão
#Se retweeted_status estiver num tweet em parsed_tweets retorne este tweet
retweetados = [tweet for tweet in parsed_tweets if 'retweeted_status' in tweet]

In [ ]:
len(retweetados)

In [ ]:
retweetados[1]

In [ ]:
first_retweet = retweetados[1]

In [ ]:
#Pegar a info do tweet original
first_retweet['retweeted_status']['created_at']

In [ ]:
first_retweet['retweeted_status']['entities']['user_mentions']

In [ ]:
# Filtrar apenas os user_mentions
entities = [entitie for entitie in retweetados if 'entities' in entitie]

In [ ]:
len(entities)

In [ ]:
entities[0]

In [ ]:
# Lição de casa!
mentions = [mention for mention in entities if 'user_mentions' in mention]

# Transformar os tweets em uma estrutura relacional analisável

In [ ]:
import pandas as pd

In [97]:
# Pega um tweet que tenha feito menção a duas ou mais pessoas
tweet_many_mentions = [
    tweet for tweet in parsed_tweets 
    if ('retweeted_status' in tweet) and
    (len(tweet['retweeted_status']['entities']['user_mentions']) >= 2)
]
len(tweet_many_mentions)

140

In [ ]:
tweet_many_mentions[0]['entities']

In [76]:
type(tweet_many_mentions[0])

dict

In [ ]:
tweet_many_mentions[0]

In [77]:
primeiro_tweet = tweet_many_mentions[0]

In [ ]:
teste = pd.DataFrame(primeiro_tweet)
teste
# Isso nao funciona!

In [82]:
#Pega somente as primeiras colunas do json (chaves) (aquelas de nivel root)
df_tratado = pd.DataFrame(primeiro_tweet).reset_index(drop=True).iloc[:1]
df_tratado

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms
0,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,0,0,0,0,NaN,False,False,low,en,1615339001038


In [83]:
df_tratado.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'entities',
       'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'],
      dtype='object')

In [84]:
#Inplace persiste persiste o mesmo DF tratado, se nao teriamos que utilizar df = df
df_tratado.drop(columns=[
    'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'user', 'entities',
    'retweeted_status'
], inplace=True)

In [72]:
df_tratado.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'filter_level', 'lang',
       'timestamp_ms', 'quoted_status_id', 'quoted_status_id_str',
       'quoted_status', 'quoted_status_permalink', 'extended_entities',
       'possibly_sensitive'],
      dtype='object')

In [85]:
df_tratado['user_id'] = primeiro_tweet['user']['id']
df_tratado['user_id_str'] = primeiro_tweet['user']['id_str']
df_tratado['user_screen_name'] = primeiro_tweet['user']['screen_name']
df_tratado['user_location'] = primeiro_tweet['user']['location']
df_tratado['user_description'] = primeiro_tweet['user']['description']
df_tratado['user_protected'] = primeiro_tweet['user']['protected']
df_tratado['user_verified'] = primeiro_tweet['user']['verified']
df_tratado['user_followers_counts'] = primeiro_tweet['user']['followers_count']
df_tratado['user_friends_counts'] = primeiro_tweet['user']['friends_count']
df_tratado['user_created_at'] = primeiro_tweet['user']['created_at']

In [86]:
df_tratado

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,user_id,user_id_str,user_screen_name,user_location,user_description,user_protected,user_verified,user_followers_counts,user_friends_counts,user_created_at
0,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,1021393315416166400,1021393315416166400,flowerPotzzz,"Ohio, USA",🇺🇸PATRIOT🇺🇸\nCountry Girl🍻✌🌻\n🇺🇸#TrumpWon🇺🇸#Mi...,False,False,7267,7910,Mon Jul 23 13:55:17 +0000 2018


In [87]:
# Entities
primeiro_tweet['entities']

{'hashtags': [],
 'urls': [],
 'user_mentions': [{'screen_name': 'Rosiesmom_',
   'name': 'Rosie’s Mom',
   'id': 1360026694694895618,
   'id_str': '1360026694694895618',
   'indices': [3, 14]},
  {'screen_name': 'twk4usa',
   'name': 'Twk4USA',
   'id': 1325803514937872385,
   'id_str': '1325803514937872385',
   'indices': [38, 46]},
  {'screen_name': 'Rosiesmom_',
   'name': 'Rosie’s Mom',
   'id': 1360026694694895618,
   'id_str': '1360026694694895618',
   'indices': [48, 59]},
  {'screen_name': 'USAF_88',
   'name': '🇺🇸✈️USAFVet🇺🇸✈️',
   'id': 1324743040951529473,
   'id_str': '1324743040951529473',
   'indices': [61, 69]},
  {'screen_name': 'Anna_toyou',
   'name': '🌻Anna',
   'id': 1360450328492191747,
   'id_str': '1360450328492191747',
   'indices': [71, 82]},
  {'screen_name': 'AngelWarrior320',
   'name': '♥️ 𝒜𝓃𝑔𝑒𝓁𝒲𝒶𝓇𝓇𝒾𝑜𝓇𝟦𝒯𝓇𝓊𝓂𝓅 ♥️',
   'id': 1137096604207517697,
   'id_str': '1137096604207517697',
   'indices': [84, 100]},
  {'screen_name': 'BackatyouLife76',
   'name': "Juli

In [88]:
user_mentions = []

for i in range(len(primeiro_tweet['entities']['user_mentions'])):
    user_mention = primeiro_tweet['entities']['user_mentions'][i].copy()
    user_mention.pop('indices', None)
    df = pd.DataFrame(user_mention, index=[0])
    df = df.rename(columns={
        'screen_name': 'entities_screen_name',
        'name': 'entities_name',
        'id': 'entities_id',
        'id_str': 'entities_id_str'
    })
    user_mentions.append(df)

In [92]:
user_mentions[0]

,entities_screen_name,entities_name,entities_id,entities_id_str
0,Rosiesmom_,Rosie’s Mom,1360026694694895618,1360026694694895618


In [93]:
dfs = []
for i in user_mentions:
    dfs.append(
        pd.concat([df_tratado.copy(), i], axis=1)
    )

In [95]:
pd.concat(dfs, ignore_index=True)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,user_description,user_protected,user_verified,user_followers_counts,user_friends_counts,user_created_at,entities_screen_name,entities_name,entities_id,entities_id_str
0,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,🇺🇸PATRIOT🇺🇸\nCountry Girl🍻✌🌻\n🇺🇸#TrumpWon🇺🇸#Mi...,False,False,7267,7910,Mon Jul 23 13:55:17 +0000 2018,Rosiesmom_,Rosie’s Mom,1360026694694895618,1360026694694895618
1,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,🇺🇸PATRIOT🇺🇸\nCountry Girl🍻✌🌻\n🇺🇸#TrumpWon🇺🇸#Mi...,False,False,7267,7910,Mon Jul 23 13:55:17 +0000 2018,twk4usa,Twk4USA,1325803514937872385,1325803514937872385
2,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,🇺🇸PATRIOT🇺🇸\nCountry Girl🍻✌🌻\n🇺🇸#TrumpWon🇺🇸#Mi...,False,False,7267,7910,Mon Jul 23 13:55:17 +0000 2018,Rosiesmom_,Rosie’s Mom,1360026694694895618,1360026694694895618
3,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,🇺🇸PATRIOT🇺🇸\nCountry Girl🍻✌🌻\n🇺🇸#TrumpWon🇺🇸#Mi...,False,False,7267,7910,Mon Jul 23 13:55:17 +0000 2018,USAF_88,🇺🇸✈️USAFVet🇺🇸✈️,1324743040951529473,1324743040951529473
4,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,🇺🇸PATRIOT🇺🇸\nCountry Girl🍻✌🌻\n🇺🇸#TrumpWon🇺🇸#Mi...,False,False,7267,7910,Mon Jul 23 13:55:17 +0000 2018,Anna_toyou,🌻Anna,1360450328492191747,1360450328492191747
5,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,🇺🇸PATRIOT🇺🇸\nCountry Girl🍻✌🌻\n🇺🇸#TrumpWon🇺🇸#Mi...,False,False,7267,7910,Mon Jul 23 13:55:17 +0000 2018,AngelWarrior320,♥️ 𝒜𝓃𝑔𝑒𝓁𝒲𝒶𝓇𝓇𝒾𝑜𝓇𝟦𝒯𝓇𝓊𝓂𝓅 ♥️,1137096604207517697,1137096604207517697
6,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,🇺🇸PATRIOT🇺🇸\nCountry Girl🍻✌🌻\n🇺🇸#TrumpWon🇺🇸#Mi...,False,False,7267,7910,Mon Jul 23 13:55:17 +0000 2018,BackatyouLife76,Julie.DJT'sgirl,1353730398459736064,1353730398459736064
7,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,🇺🇸PATRIOT🇺🇸\nCountry Girl🍻✌🌻\n🇺🇸#TrumpWon🇺🇸#Mi...,False,False,7267,7910,Mon Jul 23 13:55:17 +0000 2018,BkHGIFTS1,BACK HOUSE,2752900142,2752900142


Vamos criar uma funcao que faz todo o tratamento do tweet e converte para um DF (o mesmo passo acima mas para todos os teewts)

In [108]:
def tweet_para_df(tweet):
    try:
        df_tratado = pd.DataFrame(tweet).reset_index(drop=True).iloc[:1]
        df_tratado.drop(columns=[
            'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'user', 'entities',
            'retweeted_status'
        ], inplace=True)
        df_tratado['user_id'] = tweet['user']['id']
        df_tratado['user_id_str'] = tweet['user']['id_str']
        df_tratado['user_screen_name'] = tweet['user']['screen_name']
        df_tratado['user_location'] = tweet['user']['location']
        df_tratado['user_description'] = tweet['user']['description']
        df_tratado['user_protected'] = tweet['user']['protected']
        df_tratado['user_verified'] = tweet['user']['verified']
        df_tratado['user_followers_counts'] = tweet['user']['followers_count']
        df_tratado['user_friends_counts'] = tweet['user']['friends_count']
        df_tratado['user_created_at'] = tweet['user']['created_at']

        user_mentions = []

        for i in range(len(tweet['entities']['user_mentions'])):
            user_mention = tweet['entities']['user_mentions'][i].copy()
            user_mention.pop('indices', None)
            df = pd.DataFrame(user_mention, index=[0])
            df = df.rename(columns={
                'screen_name': 'entities_screen_name',
                'name': 'entities_name',
                'id': 'entities_id',
                'id_str': 'entities_id_str'
            })
            user_mentions.append(df)
        dfs = []
        for i in user_mentions:
            dfs.append(
                pd.concat([df_tratado.copy(), i], axis=1)
            )

        df_final = pd.concat(dfs, ignore_index=True)
    except:
        return None
    return df_final

In [ ]:
for i in range(len(tweet_many_mentions)):
    print(i)
    tweet_para_df(tweet_many_mentions[i])

In [109]:
%%time
#%%time conta o tempo de execuao desta celular
# Vamos iterar e transformar todos os tweets
parseados = [tweet_para_df(tweet) for tweet in tweet_many_mentions]

CPU times: user 2.93 s, sys: 0 ns, total: 2.93 s
Wall time: 2.93 s


In [112]:
#Elimar posições vazias
# Retorne i para cada i em parseados se i não for nulo
parseados = [i for i in parseados if i is not None]

In [113]:
len(parseados)

140

In [114]:
tratado = pd.concat(parseados, ignore_index=True)

In [115]:
tratado

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,entities_screen_name,entities_name,entities_id,entities_id_str,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,possibly_sensitive
0,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,Rosiesmom_,Rosie’s Mom,1360026694694895618,1360026694694895618,NaN,NaN,NaN,NaN,NaN,NaN
1,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,twk4usa,Twk4USA,1325803514937872385,1325803514937872385,NaN,NaN,NaN,NaN,NaN,NaN
2,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,Rosiesmom_,Rosie’s Mom,1360026694694895618,1360026694694895618,NaN,NaN,NaN,NaN,NaN,NaN
3,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,USAF_88,🇺🇸✈️USAFVet🇺🇸✈️,1324743040951529473,1324743040951529473,NaN,NaN,NaN,NaN,NaN,NaN
4,Wed Mar 10 01:16:41 +0000 2021,1369457143867465728,1369457143867465728,RT @Rosiesmom_: Trump Loves America \n\n@twk4u...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,Anna_toyou,🌻Anna,1360450328492191747,1360450328492191747,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,Wed Mar 10 01:25:06 +0000 2021,1369459264654245888,1369459264654245888,RT @ZaleskiLuke: @GOPLeader @JohnCornyn first ...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,GOPLeader,Kevin McCarthy,19739126,19739126,NaN,NaN,NaN,NaN,NaN,NaN
554,Wed Mar 10 01:25:06 +0000 2021,1369459264654245888,1369459264654245888,RT @ZaleskiLuke: @GOPLeader @JohnCornyn first ...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,JohnCornyn,Senator John Cornyn,13218102,13218102,NaN,NaN,NaN,NaN,NaN,NaN
555,Wed Mar 10 01:25:06 +0000 2021,1369459265065394185,1369459265065394185,RT @Logic_Triumphs: @BillyBaldwin Everything @...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,Logic_Triumphs,LogicTriumphs.com,3887013074,3887013074,NaN,NaN,NaN,NaN,NaN,NaN
556,Wed Mar 10 01:25:06 +0000 2021,1369459265065394185,1369459265065394185,RT @Logic_Triumphs: @BillyBaldwin Everything @...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,BillyBaldwin,Billy Baldwin,784890784273215488,784890784273215488,NaN,NaN,NaN,NaN,NaN,NaN
